In [1]:
import os
import cv2

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from concurrent.futures import ThreadPoolExecutor

In [2]:
# persiapan data
# membaca, mengubah gambar menjadi grayscale, meresize, dan mengekstrak fitur.

path_datasets = '../dataset/training_set/'
categories = ['dogs', 'cats']


def process_img(file_path, img_size=(64, 64)):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # membaca gambar sebagai grayscale
    if img is not None:
        # img_gray = cv2.cvtColor(file_path, cv2.COLOR_RGB2GRAY)
        img_resized = cv2.resize(img, img_size) # merize gambar
        return img_resized.flatten() # mengubah ke bentuk 1D
    return None

def load_gambar(images):
    file_paths = []
    labels = []
    
    for index, category in enumerate(images):
        folder_path = os.path.join(path_datasets, category)
        for file in os.listdir(folder_path):
            file_paths.append(os.path.join(folder_path, file))
            labels.append(index)

    # Memproses gambar menggunakan multithreading
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_img, file_paths))

     # Menghapus hasil yang None (gambar gagal diproses)
    processed_images = [img for img in results if img is not None]
    filtered_labels = [label for img, label in zip(results, labels) if img is not None]

    return np.array(processed_images), np.array(filtered_labels)

X, y = load_gambar(categories)

In [3]:
print(X)
print(y)

[[110 122 101 ... 240 239 233]
 [114 100  97 ...  93  91  93]
 [187 183 180 ...  62  61  67]
 ...
 [186 102 173 ... 140 132 106]
 [233 232 235 ... 232 235 230]
 [155 162 166 ... 178 152 132]]
[0 0 0 ... 1 1 1]


In [4]:
# Normalisasi nilai piksel
minMaxScaler = MinMaxScaler()
minMax = minMaxScaler.fit_transform(X)

minMax

array([[0.43137255, 0.47843137, 0.39607843, ..., 0.94117647, 0.9372549 ,
        0.91372549],
       [0.44705882, 0.39215686, 0.38039216, ..., 0.36470588, 0.35686275,
        0.36470588],
       [0.73333333, 0.71764706, 0.70588235, ..., 0.24313725, 0.23921569,
        0.2627451 ],
       ...,
       [0.72941176, 0.4       , 0.67843137, ..., 0.54901961, 0.51764706,
        0.41568627],
       [0.91372549, 0.90980392, 0.92156863, ..., 0.90980392, 0.92156863,
        0.90196078],
       [0.60784314, 0.63529412, 0.65098039, ..., 0.69803922, 0.59607843,
        0.51764706]])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(minMax, y, test_size=0.4, random_state=64)

In [6]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [7]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi: {accuracy * 100:.2f}%')

Akurasi: 62.90%
